In [2]:
from __future__ import print_function
import six
import sys
import os
from optparse import OptionParser
from keras.models import load_model, Model, model_from_json
from argparse import ArgumentParser
from keras import backend as K
from keras.layers import Input
import numpy as np
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)
import h5py
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import pandas as pd
from keras.optimizers import Adam, Nadam
from callbacks import all_callbacks
import pandas as pd
from keras.layers import Input
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import yaml
from train import parse_config, get_features

Using TensorFlow backend.


In [3]:
from collections import namedtuple
Option = namedtuple("MyStruct", "inputFile tree config outputDir")

options = Option(
    inputFile = '../data/processed-pythia82-lhc13-all-pt1-50k-r1_h022_e0175_t220_nonu_withPars_truth.z',
    tree = 't_allpar_new',
    config = 'train_config_gru.yml',
    outputDir = '../gru_weights/'
)

print("Loading configuration from", options.config)
config = open(options.config, 'r')
yamlConfig =  yaml.load(config, Loader=yaml.FullLoader)
print(yamlConfig)

Loading configuration from train_config_gru.yml
{'Inputs': ['j1_ptrel', 'j1_etarot', 'j1_phirot', 'j1_erel', 'j1_deltaR', 'j1_pdgid', 'j_index'], 'Labels': ['j_g', 'j_q', 'j_w', 'j_z', 'j_t', 'j_index'], 'KerasModel': 'gru_model', 'KerasModelRetrain': 'gru_model_constraint', 'KerasLoss': 'categorical_crossentropy', 'L1Reg': 0.0001, 'L1RegR': 0.001, 'NormalizeInputs': 1, 'InputType': 'Conv1D', 'MaxParticles': 20}


In [4]:
X_train_val, X_test, y_train_val, y_test, labels  = get_features(options, yamlConfig)

Keys: <KeysViewHDF5 ['t_allpar_new']>
load 24816
load 0
load 1000
load 2000
load 3000
load 4000
load 5000
load 6000
load 7000
load 8000
load 9000
load 10000
load 11000
load 12000
load 13000
load 14000
load 15000
load 16000
load 17000
load 18000
load 19000
load 20000
load 21000
load 22000
load 23000
load 24000
(24816, 20, 6)
(19852, 20, 6)
(4964, 20, 6)


In [8]:
from models.models import gru_model
keras_model = gru_model(Input(shape=X_train_val.shape[1:]), y_train_val.shape[1], l1Reg=yamlConfig['L1Reg'] )

startlearningrate=0.0001
adam = Adam(lr=startlearningrate)
keras_model.compile(optimizer=adam, loss=[yamlConfig['KerasLoss']], metrics=['accuracy'])


callbacks=all_callbacks(stop_patience=1000, 
                        lr_factor=0.5,
                        lr_patience=10,
                        lr_epsilon=0.000001, 
                        lr_cooldown=2, 
                        lr_minimum=0.0000001,
                        outputDir=options.outputDir)

keras_model.fit(X_train_val, y_train_val, batch_size = 1024, epochs = 100,
                validation_split = 0.25, shuffle = True, callbacks = callbacks.callbacks)



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 20, 6)             0         
_________________________________________________________________
gru_selu (GRU)               (None, 20)                1620      
_________________________________________________________________
dense_relu (Dense)           (None, 20)                420       
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
_________________________________________________________________
rnn_densef (Dense)           (None, 5)                 105       
Total params: 2,145
Trainable params: 2,145
Non-trainable params: 0
_________________________________________________________________
None


Instructions for

14889/14889 [==============================] - 0s 25us/step - loss: 1.3515 - acc: 0.7909 - val_loss: 1.3276 - val_acc: 0.8140

***callbacks***
saving losses to ../gru_weights/losses.log

Epoch 00007: val_loss improved from 1.35798 to 1.32756, saving model to ../gru_weights//KERAS_check_best_model.h5

Epoch 00007: val_loss improved from 1.35798 to 1.32756, saving model to ../gru_weights//KERAS_check_best_model_weights.h5

Epoch 00007: saving model to ../gru_weights//KERAS_check_model_last.h5

Epoch 00007: saving model to ../gru_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 8/100
14889/14889 [==============================] - 0s 26us/step - loss: 1.3209 - acc: 0.8027 - val_loss: 1.2968 - val_acc: 0.8156

***callbacks***
saving losses to ../gru_weights/losses.log

Epoch 00008: val_loss improved from 1.32756 to 1.29679, saving model to ../gru_weights//KERAS_check_best_model.h5

Epoch 00008: val_loss improved from 1.32756 to 1.29679, saving model to ../gru_weights//

Epoch 00020: val_loss improved from 0.92524 to 0.88806, saving model to ../gru_weights//KERAS_check_best_model.h5

Epoch 00020: val_loss improved from 0.92524 to 0.88806, saving model to ../gru_weights//KERAS_check_best_model_weights.h5

Epoch 00020: saving model to ../gru_weights//KERAS_check_model_last.h5

Epoch 00020: saving model to ../gru_weights//KERAS_check_model_last_weights.h5

Epoch 00020: saving model to ../gru_weights//KERAS_check_model_epoch20.h5

***callbacks end***

Epoch 21/100
14889/14889 [==============================] - 0s 20us/step - loss: 0.8828 - acc: 0.8083 - val_loss: 0.8506 - val_acc: 0.8090

***callbacks***
saving losses to ../gru_weights/losses.log

Epoch 00021: val_loss improved from 0.88806 to 0.85057, saving model to ../gru_weights//KERAS_check_best_model.h5

Epoch 00021: val_loss improved from 0.88806 to 0.85057, saving model to ../gru_weights//KERAS_check_best_model_weights.h5

Epoch 00021: saving model to ../gru_weights//KERAS_check_model_last.h5

Epoc

Epoch 00033: saving model to ../gru_weights//KERAS_check_model_last.h5

Epoch 00033: saving model to ../gru_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 34/100
14889/14889 [==============================] - 0s 24us/step - loss: 0.4494 - acc: 0.8145 - val_loss: 0.4200 - val_acc: 0.8205

***callbacks***
saving losses to ../gru_weights/losses.log

Epoch 00034: val_loss improved from 0.44359 to 0.42004, saving model to ../gru_weights//KERAS_check_best_model.h5

Epoch 00034: val_loss improved from 0.44359 to 0.42004, saving model to ../gru_weights//KERAS_check_best_model_weights.h5

Epoch 00034: saving model to ../gru_weights//KERAS_check_model_last.h5

Epoch 00034: saving model to ../gru_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 35/100
14889/14889 [==============================] - 0s 32us/step - loss: 0.4264 - acc: 0.8154 - val_loss: 0.3991 - val_acc: 0.8239

***callbacks***
saving losses to ../gru_weights/losses.log

Epoch 00035: val

14889/14889 [==============================] - 1s 39us/step - loss: 0.3084 - acc: 0.8582 - val_loss: 0.2995 - val_acc: 0.8719

***callbacks***
saving losses to ../gru_weights/losses.log

Epoch 00047: val_loss improved from 0.30334 to 0.29948, saving model to ../gru_weights//KERAS_check_best_model.h5

Epoch 00047: val_loss improved from 0.30334 to 0.29948, saving model to ../gru_weights//KERAS_check_best_model_weights.h5

Epoch 00047: saving model to ../gru_weights//KERAS_check_model_last.h5

Epoch 00047: saving model to ../gru_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 48/100
14889/14889 [==============================] - 1s 40us/step - loss: 0.3047 - acc: 0.8645 - val_loss: 0.2958 - val_acc: 0.8771

***callbacks***
saving losses to ../gru_weights/losses.log

Epoch 00048: val_loss improved from 0.29948 to 0.29578, saving model to ../gru_weights//KERAS_check_best_model.h5

Epoch 00048: val_loss improved from 0.29948 to 0.29578, saving model to ../gru_weights/


Epoch 00060: val_loss improved from 0.25657 to 0.25273, saving model to ../gru_weights//KERAS_check_best_model_weights.h5

Epoch 00060: saving model to ../gru_weights//KERAS_check_model_last.h5

Epoch 00060: saving model to ../gru_weights//KERAS_check_model_last_weights.h5

Epoch 00060: saving model to ../gru_weights//KERAS_check_model_epoch60.h5

***callbacks end***

Epoch 61/100
14889/14889 [==============================] - 1s 34us/step - loss: 0.2557 - acc: 0.9128 - val_loss: 0.2487 - val_acc: 0.9142

***callbacks***
saving losses to ../gru_weights/losses.log

Epoch 00061: val_loss improved from 0.25273 to 0.24867, saving model to ../gru_weights//KERAS_check_best_model.h5

Epoch 00061: val_loss improved from 0.25273 to 0.24867, saving model to ../gru_weights//KERAS_check_best_model_weights.h5

Epoch 00061: saving model to ../gru_weights//KERAS_check_model_last.h5

Epoch 00061: saving model to ../gru_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 62/100
1488


Epoch 00073: saving model to ../gru_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 74/100
14889/14889 [==============================] - 0s 33us/step - loss: 0.1933 - acc: 0.9248 - val_loss: 0.1818 - val_acc: 0.9170

***callbacks***
saving losses to ../gru_weights/losses.log

Epoch 00074: val_loss improved from 0.18630 to 0.18177, saving model to ../gru_weights//KERAS_check_best_model.h5

Epoch 00074: val_loss improved from 0.18630 to 0.18177, saving model to ../gru_weights//KERAS_check_best_model_weights.h5

Epoch 00074: saving model to ../gru_weights//KERAS_check_model_last.h5

Epoch 00074: saving model to ../gru_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 75/100
14889/14889 [==============================] - 1s 58us/step - loss: 0.1867 - acc: 0.9279 - val_loss: 0.1774 - val_acc: 0.9194

***callbacks***
saving losses to ../gru_weights/losses.log

Epoch 00075: val_loss improved from 0.18177 to 0.17739, saving model to ../gru_weights/

14889/14889 [==============================] - 1s 40us/step - loss: 0.1435 - acc: 0.9485 - val_loss: 0.1297 - val_acc: 0.9520

***callbacks***
saving losses to ../gru_weights/losses.log

Epoch 00087: val_loss improved from 0.13254 to 0.12969, saving model to ../gru_weights//KERAS_check_best_model.h5

Epoch 00087: val_loss improved from 0.13254 to 0.12969, saving model to ../gru_weights//KERAS_check_best_model_weights.h5

Epoch 00087: saving model to ../gru_weights//KERAS_check_model_last.h5

Epoch 00087: saving model to ../gru_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 88/100
14889/14889 [==============================] - 1s 37us/step - loss: 0.1386 - acc: 0.9501 - val_loss: 0.1269 - val_acc: 0.9561

***callbacks***
saving losses to ../gru_weights/losses.log

Epoch 00088: val_loss improved from 0.12969 to 0.12692, saving model to ../gru_weights//KERAS_check_best_model.h5

Epoch 00088: val_loss improved from 0.12969 to 0.12692, saving model to ../gru_weights/

Epoch 00100: val_loss improved from 0.10730 to 0.10614, saving model to ../gru_weights//KERAS_check_best_model_weights.h5

Epoch 00100: saving model to ../gru_weights//KERAS_check_model_last.h5

Epoch 00100: saving model to ../gru_weights//KERAS_check_model_last_weights.h5

Epoch 00100: saving model to ../gru_weights//KERAS_check_model_epoch100.h5

***callbacks end***



In [9]:
with open("../gru_weights/model.json", "w") as json_file:
    json_file.write(keras_model.to_json())